In [38]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import requests

In [39]:
U_df=pd.read_csv('data/U.csv')
V_T_df=pd.read_csv('data/V_T.csv')
V_inv_df=pd.read_csv('data/V_inv.csv')

In [40]:
U=np.array(U_df)
V=np.array(V_T_df).T
V_inv=np.array(V_inv_df)

In [41]:
V_inv.shape

(99217, 10)

In [42]:
url='https://anilist.co/api/'
cid='selzla-6acux'
sec='eGi4fmsY9pV64E1fSTWJJ1'
username='Goldensunboy'
params={'grant_type':"client_credentials",'client_id':cid,'client_secret':sec}
access=requests.post(url+'auth/access_token',data=params)
access_token=access.json()['access_token']

In [43]:
user_anime=requests.get(url+'user/'+username+'/animelist?access_token='+access_token)

In [44]:
completed = user_anime.json()['lists']['completed']

In [45]:
scores = []
for i in range(len(completed)):
    title=completed[i]['anime']['title_english'].encode("utf-8")
    anime_id=completed[i]['anime']['id']
    score=completed[i]['score_raw']
    scores.append([anime_id, score])

In [46]:
series_df=pd.read_csv('data/series_data.csv')

In [47]:
titles={}
for i in range(len(series_df)):
    ID = series_df['id'][i]
    title = series_df['title_english'][i]
    titles[ID] = title


In [48]:
user=np.array([row for row in scores if row[0] in np.array(series_df['id'])])
user=np.array([[row[0]-1, row[1]] for row in user])
# print user

In [53]:
# np.array([[row[0]-1, row[1]] for row in user])

In [50]:
user_vector = np.zeros(V_inv.shape[0])
for row in user:
    user_vector[row[0]]=row[1]

In [51]:
new_decomp=np.dot(user_vector, V_inv)
new_R=np.dot(new_decomp, V)

In [54]:
seen = [row[0] for row in scores]
rec=np.argsort(-1*new_R)
count = 0
for i in range(100):
    if rec[i]+1 not in seen:
        print rec[i]+1, titles[rec[i]+1]
        count +=1 
    if count>10:
        break
    
    

6547 Angel Beats!
11757 Sword Art Online
11741 Fate/Zero 2nd Season
5081 Bakemonogatari
4181 Clannad After Story
2167 Clannad
4224 Toradora!
9989 Anohana: The Flower We Saw That Day
11597 Nisemonogatari
11111 Another
30 Neon Genesis Evangelion
